In [3]:
!pip install spektral

     |████████████████████████████████| 122kB 5.5MB/s 


In [16]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import ChebConv
from spektral.transforms import AdjToSpTensor, LayerPreprocess

# Load data
dataset = Citation("cora", transforms=[LayerPreprocess(ChebConv), AdjToSpTensor()])




# We convert the binary masks to sample weights so that we can compute the
# average loss over the nodes (following original implementation by
# Kipf & Welling)
def mask_to_weights(mask):
    return mask / np.count_nonzero(mask)


weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

# Parameters
channels = 16  # Number of channels in the first layer
K = 2  # Max degree of the Chebyshev polynomials
dropout = 0.5  # Dropout rate for the features
l2_reg = 2.5e-4  # L2 regularization rate
learning_rate = 1e-2  # Learning rate
epochs = 200  # Number of training epochs
patience = 10  # Patience for early stopping
a_dtype = dataset[0].a.dtype  # Only needed for TF 2.1

N = dataset.n_nodes  # Number of nodes in the graph
F = dataset.n_node_features  # Original size of node features
n_out = dataset.n_labels  # Number of classes




# # Model definition
# x_in = Input(shape=(F,))
# a_in = Input((N,), sparse=True, dtype=a_dtype)

# do_1 = Dropout(dropout)(x_in)
# gc_1 = ChebConv(
#     channels, K=K, activation="relu", kernel_regularizer=l2(l2_reg), use_bias=False
# )([do_1, a_in])
# do_2 = Dropout(dropout)(gc_1)
# gc_2 = ChebConv(n_out, K=K, activation="softmax", use_bias=False)([do_2, a_in])

# # Build model
# model = Model(inputs=[x_in, a_in], outputs=gc_2)
# optimizer = Adam(lr=learning_rate)
# model.compile(
#     optimizer=optimizer,
#     loss=CategoricalCrossentropy(reduction="sum"),  # To compute mean
#     weighted_metrics=["acc"],
# )
# model.summary()

# # Train model
# loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
# loader_va = SingleLoader(dataset, sample_weights=weights_va)
# model.fit(
#     loader_tr.load(),
#     steps_per_epoch=loader_tr.steps_per_epoch,
#     validation_data=loader_va.load(),
#     validation_steps=loader_va.steps_per_epoch,
#     epochs=epochs,
#     callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
# )

# # Evaluate model
# print("Evaluating model.")
# loader_te = SingleLoader(dataset, sample_weights=weights_te)
# eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
# print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [17]:
import sys
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from math import ceil

import warnings
warnings.filterwarnings('ignore')

In [18]:
def get_graph_features(G, all_embds, nmax=15):

    n = len(G.nodes())
    
    node2id = {node:i for i, node in enumerate(G.nodes())}
    id2node = {i:node for node,i in node2id.items()}

    adj = np.zeros((nmax,nmax))
    embds = np.zeros((nmax, all_embds.shape[1]))

    for i in G.nodes():
        embds[node2id[i]] = all_embds[i]
        for j in G.neighbors(i):
            adj[node2id[j],node2id[i]] = 1
    
    return adj, embds

In [19]:
def simulate_graph(nb_samples=50, vocab_size=15, graph_length=10, 
                   word_embed_dim=3, nb_classes=5, probas = None, 
                   centers=None, random_state=None):
    
    np.random.seed(random_state)
    
    graphs = []
    
    y = np.zeros(nb_samples)
    
    for i in range(nb_samples):
        
        p = np.random.uniform()
        
        cat = int(p*nb_classes)
        
        y[i] = cat
        
        if probas is None:
        
            G = nx.binomial_graph(graph_length, p)
        else:
            
            G = nx.binomial_graph(graph_length, probas[cat])
        
        new_nodes = np.random.randint(vocab_size*cat,vocab_size*(cat+1), graph_length).tolist()
        
        mapping  = dict(zip(G.nodes(), new_nodes))
        
        G = nx.relabel_nodes(G,mapping)
        
        graphs.append(G)

    try:
        embds = np.vstack((c + np.random.normal(size = (vocab_size, word_embed_dim))  for c in centers))
    except:
        embds = np.random.normal(size = (nb_classes *  vocab_size, word_embed_dim))
    
    return embds, graphs, y